In [32]:
import torch
import torch.nn as nn
import dgl
import networkx as nx
from math import log
from torch.nn import init

In [33]:
# creating sample graph!!
Gn = nx.karate_club_graph()

# converting networkx to dgl graph
G = dgl.DGLGraph()
G.from_networkx(Gn)

In [34]:
# creating nodes and edgelist for random process!!
nodes = list(G.nodes())
edgedict = {}
for node in nodes:
    edgedict[node] = G.successors(node)

In [35]:
# random sampling of node and edges 
import random
random.seed(8233232432)

def sample_neighbor(node,alpha):
    dval = random.uniform(0,1)
    if dval < alpha:
        v = random.choice(edgedict[node])
    else:
        v = node
    return v

random.shuffle(nodes) 
u_node = random.choice(nodes) # sampling a node
v_node = sample_neighbor(u_node,0.85) # sampling positive example
v_neg_node = random.choice(nodes) # sampling negative example
print(u_node,v_node,v_neg_node)

tensor(28) tensor(2) tensor(23)


In [36]:
class Verse(nn.Module): # says we are building a custom neural network!!
    
    def __init__(self,nodes,edges,embedding,epochs,neg,lr,alpha):
        super(Verse, self).__init__()
        
        self.nodes = nodes
        self.edges = edges
        self.num_nodes = len(nodes)
        self.embedding = embedding
        self.nce_bias = log(self.num_nodes)
        self.nce_neg_bias = log(self.num_nodes/neg)
        self.training_count = len(nodes) * epochs
        self.neg = neg
        self.lr = lr
        self.alpha = alpha
        
        # intializing device
        self.device = torch.device("cpu")
        
        # Weight matrix
        initrange = 0.5
        self.W = nn.Embedding(self.num_nodes, self.embedding, sparse=True) 
        init.uniform_(self.W.weight.data, -initrange, initrange)
        
        # sigmoid table
        self.lookup_table = torch.sigmoid(torch.arange(-6.01, 6.01, 0.01))
        self.lookup_table[0] = 0.
        self.lookup_table[-1] = 1.
    
    def set_device(self, gpu_id):
        """ set gpu device """
        self.device = torch.device("cuda:%d" % gpu_id)
        print("The device is", self.device)
        self.lookup_table = self.lookup_table.to(self.device)
        self.node = self.node.to(self.device)
        self.edges = self.edges.to(self.device)
    
    def share_memory(self):
        """ share the parameters across subprocesses """
        self.W.weight.share_memory_()
    
    def fast_sigmoid(self, score):
        """ do fast sigmoid by looking up in a pre-defined table """
        idx = torch.floor((score + 6.01) / 0.01).long()
        return self.lookup_table[idx]

    def train(self):
        lr = self.lr
        nce_bias = self.nce_bias
        nce_neg_bias = self.nce_neg_bias
        neg = self.neg
        alpha = self.alpha # alpha value to return to start node.
        count = 0
        while True:
            if (count > self.training_count):
                break
            u_node = random.choice(self.nodes) # sample a node
            v_node = sample_neighbor(u_node,alpha) # sample a positive node
            self.update(u_node,v_node, 1, nce_bias, lr) # update weight for positive example
            for i in range(neg):
                v_neg_node = random.choice(self.nodes) # sample a negative node
                self.update(u_node,v_neg_node, 0, nce_neg_bias, lr) # update weight for negative example
            count += 1
        return
    
    def update(self, u, v, label, bias, lr):
        score = -bias
        W_u = self.W.weight[u]
        W_v = self.W.weight[v]
        score += torch.matmul(W_u,W_v)
        score = torch.clamp(score, max=6, min=-6)
        score = (label - self.fast_sigmoid(score)) * lr
        self.W.weight[u] += W_v * score
        self.W.weight[v] += W_u * score
        return

    def save_embedding_txt(self, file_name):
        embedding = self.W.weight.cpu().data.numpy()
        with open(file_name, 'w') as f:
            f.write('%d %d\n' % (self.num_nodes, self.embedding))
            for wid in range(self.num_nodes):
                e = ' '.join(map(lambda x: str(x), embedding[wid]))
                f.write('%s\n' % (e))

In [37]:
verse = Verse(nodes,edgedict,128,2,3,0.0025,0.85)

In [38]:
verse.train()

In [39]:
verse.save_embedding_txt('C:/Users/ajay madhavan/Documents/verse-master/test_emb.txt')